<h2 align=center> Facial Expression Recognition with Keras</h2>

### Task 1: Import Libraries

In [2]:
!pip install utils

In [3]:
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import utils
import os
%matplotlib inline

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense, Input, Dropout,Flatten, Conv2D
from tensorflow.keras.layers import BatchNormalization, Activation, MaxPooling2D
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.utils import plot_model

from IPython.display import SVG, Image
import tensorflow as tf
print("Tensorflow version:", tf.__version__)

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


Tensorflow version: 2.2.0-rc3


In [0]:
!pip install -q kaggle
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!ls ~/.kaggle
!chmod 600 /root/.kaggle/kaggle.json  # set permission
!pip install --upgrade --force-reinstall -- kaggle
!kaggle competitions download -c challenges-in-representation-learning-facial-expression-recognition-challenge -p /content/



In [0]:
import os
os.chdir('../content/')  #change dir
!unzip -q challenges-in-representation-learning-facial-expression-recognition-challenge.zip 

In [0]:
!tar -xf fer2013.tar.gz

In [0]:
os.mkdir('/content/Tr')
os.mkdir('/content/Tr/0')
os.mkdir('/content/Tr/1')
os.mkdir('/content/Tr/2')
os.mkdir('/content/Tr/3')
os.mkdir('/content/Tr/4')
os.mkdir('/content/Tr/5')
os.mkdir('/content/Tr/6')
os.mkdir('/content/Vl')
os.mkdir('/content/Vl/0')
os.mkdir('/content/Vl/1')
os.mkdir('/content/Vl/2')
os.mkdir('/content/Vl/3')
os.mkdir('/content/Vl/4')
os.mkdir('/content/Vl/5')
os.mkdir('/content/Vl/6')

In [0]:
import numpy as np
import cv2
import os



with open("/content/fer2013/fer2013.csv") as f:
    content = f.readlines()

lines = np.array(content)

num_of_instances = lines.size

for i in range(1,num_of_instances):
    try:
        emotion, img, usage = lines[i].split(",")
        img = img.replace('"', '')
        img = img.replace('\n', '')
        pixels = img.split(" ")
        
        pixels = np.array(pixels, 'float32')
        image = pixels.reshape(48, 48)
        if usage == 'Training\n':
          if emotion == "0":
            path_file_name = f"/content/Tr/0/Training_{i}.jpg"

          if emotion == "1":
            path_file_name = f"/content/Tr/1/Training_{i}.jpg"

          if emotion == "2":
            path_file_name = f"/content/Tr/2/Training_{i}.jpg"

          if emotion == "3":
            path_file_name = f"/content/Tr/3/Training_{i}.jpg"

          if emotion == "4":
            path_file_name = f"/content/Tr/4/Training_{i}.jpg"

          if emotion == "5":
            path_file_name = f"/content/Tr/5/Training_{i}.jpg"

          if emotion == "6":
            path_file_name = f"/content/Tr/6/Training_{i}.jpg"

        if usage == 'PublicTest\n':
          if emotion == "0":
            path_file_name = f"/content/Vl/0/PublicTest_{i}.jpg"

          if emotion == "1":
            path_file_name = f"/content/Vl/1/PublicTest_{i}.jpg"

          if emotion == "2":
            path_file_name = f"/content/Vl/2/PublicTest_{i}.jpg"

          if emotion == "3":
            path_file_name = f"/content/Vl/3/PublicTest_{i}.jpg"

          if emotion == "4":
            path_file_name = f"/content/Vl/4/PublicTest_{i}.jpg"

          if emotion == "5":
            path_file_name = f"/content/Vl/5/PublicTest{i}.jpg"

          if emotion == "6":
            path_file_name = f"/content/Vl/6/PublicTest_{i}.jpg"

        cv2.imwrite(path_file_name, image)

    except Exception as ex:
        print(ex)


### Task 2: Samples

In [9]:
for expression in os.listdir("/content/Tr"):
  print(str(len(os.listdir("/content/Tr/" + expression))) + " " + expression + "_images")

4965 6_images
3171 5_images
4830 4_images
3995 0_images
7215 3_images
436 1_images
4097 2_images


### Task 3: Generate Training and Validation Batches

In [10]:
img_size = 48
batch_size = 64

datagen_train = ImageDataGenerator(horizontal_flip= True)
train_generator = datagen_train.flow_from_directory('/content/Tr', target_size = (img_size, img_size),
                                                    color_mode = 'grayscale',
                                                    batch_size = batch_size,
                                                    class_mode = 'categorical',
                                                    shuffle = True)
datagen_val = ImageDataGenerator(horizontal_flip= True)
val_generator = datagen_val.flow_from_directory('/content/Vl', target_size = (img_size, img_size),
                                                    color_mode = 'grayscale',
                                                    batch_size = batch_size,
                                                    class_mode = 'categorical',
                                                    shuffle = True)

Found 28709 images belonging to 7 classes.
Found 3589 images belonging to 7 classes.


### Task 4: Create CNN Model

In [11]:
model = Sequential()
model.add(Conv2D(64,(3,3), padding = 'same', input_shape = (48,48,1)))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size = (2,2)))
model.add(Dropout(0.25))

model.add(Conv2D(128,(5,5), padding = 'same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size = (2,2)))
model.add(Dropout(0.25))

model.add(Conv2D(512,(3,3), padding = 'same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size = (2,2)))
model.add(Dropout(0.25))

model.add(Conv2D(512,(3,3), padding = 'same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size = (2,2)))
model.add(Dropout(0.25))

model.add(Flatten())

model.add(Dense(256))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.25))

model.add(Dense(512))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.25))

model.add(Dense(7, activation='softmax'))

optimizer = Adam(lr = 0.0005)
model.compile(optimizer= optimizer, loss = 'categorical_crossentropy', metrics = ['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 48, 48, 64)        640       
_________________________________________________________________
batch_normalization (BatchNo (None, 48, 48, 64)        256       
_________________________________________________________________
activation (Activation)      (None, 48, 48, 64)        0         
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 24, 24, 64)        0         
_________________________________________________________________
dropout (Dropout)            (None, 24, 24, 64)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 24, 24, 128)       204928    
_________________________________________________________________
batch_normalization_1 (Batch (None, 24, 24, 128)       5

### Task 6: Train and Evaluate Model

In [13]:
epochs = 30
steps_per_epoch = train_generator.n//train_generator.batch_size
validation_steps = val_generator.n//train_generator.batch_size

checkpoint = ModelCheckpoint('model_weights.h5', monitor = 'val_accuracy', 
                             save_weights_only = True, mode = 'max', verbose = 1)

reduce_lr = ReduceLROnPlateau(monitor = 'val_loss', factor = 0.1, patience= 2, 
                              min_lr = 0.00001, model = 'auto')

callbacks = [checkpoint, reduce_lr]

history = model.fit(
    x =train_generator,
    epochs = epochs,
    steps_per_epoch = steps_per_epoch,
    validation_data = val_generator,
    validation_steps = validation_steps,
    callbacks = callbacks
)

Epoch 1/30
447/448 [============================>.] - ETA: 0s - loss: 1.3088 - accuracy: 0.4969
Epoch 00001: saving model to model_weights.h5
448/448 [==============================] - 19s 42ms/step - loss: 1.3089 - accuracy: 0.4970 - val_loss: 1.3374 - val_accuracy: 0.4785 - lr: 5.0000e-04
Epoch 2/30
447/448 [============================>.] - ETA: 0s - loss: 1.2255 - accuracy: 0.5321
Epoch 00002: saving model to model_weights.h5
448/448 [==============================] - 19s 42ms/step - loss: 1.2254 - accuracy: 0.5322 - val_loss: 1.2276 - val_accuracy: 0.5382 - lr: 5.0000e-04
Epoch 3/30
447/448 [============================>.] - ETA: 0s - loss: 1.1859 - accuracy: 0.5483
Epoch 00003: saving model to model_weights.h5
448/448 [==============================] - 19s 42ms/step - loss: 1.1856 - accuracy: 0.5484 - val_loss: 1.2898 - val_accuracy: 0.4978 - lr: 5.0000e-04
Epoch 4/30
447/448 [============================>.] - ETA: 0s - loss: 1.1409 - accuracy: 0.5640
Epoch 00004: saving model to